# **5ED3S Mini Project #9**
## Motivation
I have been working on this kernel as part of the 5th European Data Science Summer School. In addition to developing a classification model for the dataset, the aim is also to develop a business model whose core element is the solution to the machine learning problem.

The problem is to classify whether a person earns more or less than 50k. As data source the [UCI Adult Data Set](http://archive.ics.uci.edu/ml/datasets/Adult) is used which is also available for import on Kaggle.

This kernel is structured in five parts:
1. Gathering Data
2. Data Exploration
3. Developing a Business Modell
4. Data Cleaning
5. Model the Data
6. Conclusion

# 1. Gathering Data
## 1.1 The Dataset

The dataset was created in 1994 by Barry Becker. It is an extraction of the census database of the [United States Census Bureus](https://www.census.gov/en.html) from 1994. Together with Ronny Kohavi, Becker published the dataset in 1996.

## 1.2 Get the Data
Let's start with checking the pyhton version and importing the data.

In [ ]:
#Start with importing system and operating system libraries
import sys
import os

#Suppress warnings, this will not affect the result
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

#Check python version
print('Python version:',sys.version)

#Import pandas and numpy
import pandas as pd
import numpy as np

#Load the data set
data = pd.read_csv('../input/adult.csv', sep=',')
data.head()

# 2. Data Exploration
As seen above, there are already some missing values in the first rows of the data set. So let's look at the relative amount of missing data in the dataset and see if it has an impact.

In [ ]:
import matplotlib.pyplot as plt

#Initialize an empty array to collect the sum of missing values per column
mvs = []

#Count the missing values
for x in data.columns:
    mvs.append(data[x].isin(["?"]).sum())

#Build the plot
fig, ax =  plt.subplots(figsize=(10,3))
index   = np.arange(data.shape[1])

ax.bar(index, mvs, alpha = 0.4, color = 'b')
ax.set_ylabel('Missing Values')
ax.set_xticks(index)
ax.set_xticklabels((data.columns))

fig.tight_layout()

plt.xticks(rotation=45)
plt.show()

In [ ]:
#Only three features contain missing values
#To see if the missing values have an significant effect on the dataset
#Visualize the missing values compared with the complete dataset to see the effect

#Build the plot
yvalues = [data.shape[0], mvs[1], mvs[6], mvs[13]]

fig, ax = plt.subplots()
index   = np.arange(4)

ax.bar(index, yvalues, alpha = 0.4, color = 'b')
ax.set_ylabel('Data')
ax.set_xticks(index)
ax.set_xticklabels(('data set size','workclass','occupation','native.country'))

fig.tight_layout()

plt.xticks(rotation=45)
plt.show()

In addition to the structural properties of the dataset, we also want to know distributions within the data. What is the average age and what is the age structure like? Are there more men or women covered by the dataset? How many hours do people work on average? What are the top 5 native countries? And how is the classification variable, income, distributed?

In [ ]:
#Question 1: Age

#Set up a histogram
plt.hist(data.age, facecolor='green', alpha=0.5, bins=18, edgecolor='black')
plt.xlabel('Age')
plt.axvline(data.age.mean(), color='red', label='average age')
plt.axis([15, 95, 0, 3500])
plt.legend()
plt.show()

The histogram acts as expected. The number of employees rises sharply at the beginning of the graph until it reaches its maximum between 30 and 45 years. This is also where the average age of 38 is. After that, the graph falls slowly. What is noticeable is that towards the end there is growth again.

In [ ]:
#Question 2: Gender

#Count male and female
m = 0
f = 0
for g in data.sex:
    if g == 'Male':
        m += 1
    if g == 'Female':
        f += 1

#Set up pie chart
colors = ['lightskyblue', 'lightcoral']
values = [m, f] 
labels = ['Male', 'Female'] 
plt.pie(values, labels=labels, colors=colors, shadow=True, startangle=90, autopct='%.2f')
plt.axis('equal')
plt.tight_layout()
plt.show()

The graph shows that more than twice as many men are represented in this dataset than women.

In [ ]:
#Question 3: Hours per Week

print('Mean:', data['hours.per.week'].mean())
#Set up a histogram
plt.hist(data['hours.per.week'], facecolor='green', alpha=0.5, bins=18, edgecolor='black')
plt.xlabel('Age')
plt.axvline(data['hours.per.week'].mean(), color='red', label='average hour per week')
plt.legend()
plt.show()

This statistic was to be expected. In the dataset there is a big peak of 40 hours per week. This is also proven by the average time of 40.43 hours per week. The increase of almost 1000 employees working in a 50 hour week should also be noted.

In [ ]:
#Question 4: Native Country

#Count the countries 
f = data['native.country'].value_counts()

#Show the top 5
f.head()

About 90% of all persons come from the USA. This is due to the fact that the data was taken from the database of the U.S. Census Bureau, so it was a census in the US.

In [ ]:
#Question 5: Income

#Count how often people earn more and less than 50k
m = 0
l = 0
for i in data.income:
    if i == '<=50K':
        l += 1
    if i == '>50K':
        m += 1

#Set up pie chart
colors = ['lightskyblue', 'lightcoral']
values = [l, m] 
labels = ['<=50k', '>50k'] 
plt.pie(values, labels=labels, colors=colors, shadow=True, startangle=90, autopct='%.2f')
plt.axis('equal')
plt.tight_layout()
plt.show()

About three quarters of employees earn less than 50k. This figure will also be important during the evaluation of our prediction model. To be usable, it must be significantly better than 75%.

# 3. Developing a Business Model

Now that you have gained an insight into the data set, you can begin to define the problem. Several Questions have to be asked. Which problem should the classification model solve? Why is it relevant to solve the problem? For whom is the solution important?

Before we devote ourselves to the benefits of problem solving, we first clarify for whom the problem will be important. This question is very easy to answer because it is an US American dataset. The information in the dataset only concerns the American market, which we were able to show in part with the distribution of the native countries. A special feature of the US market is the handling of money. Or rather, dealing with debt. In contrast to other markets, such as the German-speaking countries, credits in the US are not so ridiculed and are often the only means. An example of this would be the indebtedness that the majority of students have to afford because they would otherwise not be able to pay the high tuition fees. 

That is why credit institutions often have to ask themselves whether someone is creditworthy. And here the classification model should be a solution. By recording simple personal data, employees, without a statistical background, should be able to determine the creditworthiness of individual persons.

Now that the problem has been defined, you can proceed and take the next step developing a model.

# 4. Data Cleaning

Data cleaning is about simplifying the data. A data model is to be created which enables a machine learning algorithm to process the information optimally. For this purpose, it should be avoided to leave unnecessary data and information redundancies in the dataset.

But before we take a closer look at the individual features, we first go into the structural condition of the data. As seen above, a small part of the data has missing values. Due to the fact there is only a low percentage of missing values in the data, we can delete this rows.

In [ ]:
print('Number of rows with missing values:', data.shape[0],'\n')

#Save the number of size of the dataset before deleting the missing values
numRows = data.shape[0]

#Check if there are missing values in the dataset and delete the data if so
for x in data.columns:
    mv = data[x].isin(["?"]).sum()
    if mv > 0:
        data = data[data[x] != '?']

print('Number of rows without missing values:',data.shape[0])
print('We dropped of',numRows - data.shape[0],'rows')

#Plot a pie chart to visualize the result
colors = ['lightskyblue', 'lightcoral']
patches, texts = plt.pie([data.shape[0], numRows - data.shape[0]], colors=colors, shadow=True, startangle=90)
plt.legend(patches, ['Number of Rows', 'Number of dropped Rows'], loc="best")
plt.axis('equal')
plt.tight_layout()
plt.show()

After deleting about 2400 rows containing missing values, the data set is still big enough to build a valid prediction model on it.

## 4.1 Feature Tuning
In the next step we want to look on the attributes of each feature and identify if there are similar attributes which we can combine.

In [ ]:
#Print the attributes of workclass an their occurrence.
f = data['workclass'].value_counts().reset_index()
f.columns = ['workclass', 'count']
print(f)

Due to similarity we can combine *Self-emp-not-inc* and *Self-emp-inc* to one attribute as well as *Local-gov*, *State-gov* and *Federal-gov*.  Instead of 5, we now have 2 new attributes - *Self-emp* and *Gov*.

In [ ]:
#Combine the mentioned attributes
data.workclass = data.workclass.replace({'Self-emp-not-inc': 'Self-emp',
                                        'Self-emp-inc': 'Self-emp',
                                        'Local-gov': 'Gov',
                                        'Federal-gov': 'Gov',
                                        'State-gov': 'Gov'})

#Count all distinct attributes of fnlwgt
f = data['fnlwgt'].value_counts().reset_index()
f.columns = ['fnlwgt', 'count']
print('Number of distinct attribute in column fnlwgt :',f.shape[0])

Column *fnlwgt* has over 20000 distinct attributes. There are too many for simplification.

In [ ]:
#Print the attributes of education and their occurrence.
f = data['education'].value_counts().reset_index()
f.columns = ['education', 'count']
print('\n',f)

According to the calculation of *education*, we can combine some features. *Preschool*, *1st-4th*, *5th-6th*, *7th-8th*, *9th*, *10th*, *11th* and *12th* we summarize as *No-school*. *Some-college*, *Assoc-voc* and *Assoc-acdm* can be combined as *College*. 

In [ ]:
#Combine the mentioned attributes
data.education = data.education.replace({'Preschool': 'No-school',
                                        '1st-4th': 'No-school',
                                        '5th-6th': 'No-school',
                                        '7th-8th': 'No-school',
                                        '9th': 'No-school',
                                        '10th': 'No-school',
                                        '11th': 'No-school',
                                        '12th': 'No-school',
                                        'Some-college': 'College',
                                        'Assoc-voc': 'College',
                                        'Assoc-acdm': 'College'})

#Print the attributes of education an their occurrence
f = data['marital.status'].value_counts().reset_index()
f.columns = ['marital.status', 'count']
print('\n',f)

For reasons of simplicity, we simplify the attributes. *Married-civ-spouse* and *Married-AF-spouse* are combined as *Married*. *Never-married* and *Married-spouse-absent* are combined as *Not-married*. *Divorced* is added to *Separated*. *Widowed* remains untouched.

In [ ]:
#Combine the mentioned attributes
data['marital.status'].replace(['Married-civ-spouse'], 'Married', inplace=True)
data['marital.status'].replace('Never-married', 'Not-married', inplace=True)
data['marital.status'].replace(['Divorced'], 'Separated', inplace=True)
data['marital.status'].replace(['Separated'], 'Separated', inplace=True)
data['marital.status'].replace(['Married-spouse-absent'], 'Not-married', inplace=True)
data['marital.status'].replace(['Married-AF-spouse'], 'Married', inplace=True)

#Show the result
f = data['marital.status'].value_counts().reset_index()
f.columns = ['marital.status', 'count']
print('\n',f)

Next we look at *occupation*, *relationship*, *race* and *sex*. 

In [ ]:
#Print the attributes of occupation an their occurrence
f = data['occupation'].value_counts().reset_index()
f.columns = ['occupation', 'count']
print('\n',f)

#Print the attributes of relationship an their occurrence
f = data['relationship'].value_counts().reset_index()
f.columns = ['relationship', 'count']
print('\n',f)

#Print the attributes of race an their occurrence
f = data['race'].value_counts().reset_index()
f.columns = ['race', 'count']
print('\n',f)

#Print the attributes of sex an their occurrence
f = data['sex'].value_counts().reset_index()
f.columns = ['sex', 'count']
print('\n',f)

*occupation* is one of the important feature in the dataset, so we leave it unchanged.
*relationship*, *race* and *sex*, all have a moderate number of different attributes, which differ in their information content. This means that we don't have to change these columns.

Finally, we consider the native countries of the people.

In [ ]:
#Print the attributes of native.country an their occurrence
f = data['native.country'].value_counts().reset_index()
f.columns = ['native.country', 'count']
print('\n',f)

The *United-States* is significantly ahead of *Mexico* as native country, the second in the list, as we already saw above. The difference between the other countries is not so clear. Because there are more than 40 countries in the list, a grouping by continent makes sense. 

In [ ]:
data['native.country'].replace(['United-States'], 'N-America', inplace=True)
data['native.country'].replace(['Mexico'], 'N-America', inplace=True)
data['native.country'].replace(['Philippines'], 'Asia', inplace=True)
data['native.country'].replace(['Germany'], 'Europe', inplace=True)
data['native.country'].replace(['Puerto-Rico'], 'N-America', inplace=True)
data['native.country'].replace(['Canada'], 'N-America', inplace=True)
data['native.country'].replace(['India'], 'Asia', inplace=True)
data['native.country'].replace(['El-Salvador'], 'MS-America', inplace=True)
data['native.country'].replace(['Cuba'], 'MS-America', inplace=True)
data['native.country'].replace(['England'], 'Europe', inplace=True)
data['native.country'].replace(['Jamaica'], 'MS-America', inplace=True)
data['native.country'].replace(['Italy'], 'Europe', inplace=True)

data['native.country'].replace(['China'], 'Asia', inplace=True)
data['native.country'].replace(['Dominican-Republic'], 'MS-America', inplace=True)
data['native.country'].replace(['Vietnam'], 'Asia', inplace=True)
data['native.country'].replace(['Guatemala'], 'MS-America', inplace=True)
data['native.country'].replace(['Japan'], 'Asia', inplace=True)
data['native.country'].replace(['Columbia'], 'MS-America', inplace=True)
data['native.country'].replace(['Poland'], 'Europe', inplace=True)
data['native.country'].replace(['Taiwan'], 'Asia', inplace=True)
data['native.country'].replace(['Haiti'], 'MS-America', inplace=True)
data['native.country'].replace(['Iran'], 'Asia', inplace=True)
data['native.country'].replace(['Portugal'], 'Europe', inplace=True)
data['native.country'].replace(['Nicaragua'], 'MS-America', inplace=True)

data['native.country'].replace(['Peru'], 'MS-America', inplace=True)
data['native.country'].replace(['Greece'], 'Europe', inplace=True)
data['native.country'].replace(['Ecuador'], 'MS-America', inplace=True)
data['native.country'].replace(['France'], 'Europe', inplace=True)
data['native.country'].replace(['Ireland'], 'Europe', inplace=True)
data['native.country'].replace(['Hong'], 'Asia', inplace=True)
data['native.country'].replace(['Trinadad&Tobago'], 'MS-America', inplace=True)
data['native.country'].replace(['Cambodia'], 'Asia', inplace=True)
data['native.country'].replace(['Laos'], 'Asia', inplace=True)
data['native.country'].replace(['Thailand'], 'Asia', inplace=True)
data['native.country'].replace(['Yugoslavia'], 'Europe', inplace=True)
data['native.country'].replace(['Outlying-US(Guam-USVI-etc)'], 'N-America', inplace=True)

data['native.country'].replace(['Hungary'], 'Europe', inplace=True)
data['native.country'].replace(['Honduras'], 'MS-America', inplace=True)
data['native.country'].replace(['Scotland'], 'Europe', inplace=True)
data['native.country'].replace(['Holand-Netherlands'], 'Europe', inplace=True)

#Show the result
f = data['native.country'].value_counts().reset_index()
f.columns = ['native.country', 'count']
print('\n',f)

Now we can look at our dataset again.

In [ ]:
print('dataset size')
print('# of rows:', data.shape[0])
print('# of columns:', data.shape[1])
data.head()

The data is simplfied but there are still have 15 columns including the *income* column left. When looking at *education* and *education.num* , you can quickly see that they contain the same information. People with a high level of education have usually invested more years in their education than those with a lower level of education. Therefore, in the following we will only look at one column in our prediction model - *education*.
For further simplification, we will also remove the columns *relationship* and *fnlwgt*.  
The statement of *fnlwgt*, final weight, is very complex. To understand this, one has to know that the data set of the U.S. Census Bureaus consists of 51 individual state samples, which were collected with different methods. Persons with the same demographic characteristics should have a similar weight. However, this weight should only have significance within a state, so that we cannot take it further into account for the overall analysis ([more information about fnlwgt](http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names)). *relationship* contains information that can be partly derived from the data in *gender* and *marital.status*. Redundant information is not needed in our prediction model.  It can bias the prediction. Consequential, *relationship* is also dropped.

In [ ]:
#Drop the columns not needed
data.drop(columns=['education.num', 'fnlwgt', 'relationship'], axis=1, inplace=True)
data.head()

## 4.2 Process the Data

Now that all features are selected, we want to prepare the data. The prediction variable, *income*, is encoded binary, in 0 and 1. Numeric features are scaled with a MinMax method into a number range between 0 and 1. The remaining features are first label encoded and afterwards one hot encoded. Thus, the final data model consists of 47 attributes with a value between 0 and 1.

In [ ]:
#Import OneHotEncoder, LabelEncoder & MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

#Define how to process which feature
columns_to_label_encode   = ['income']
columns_to_scale          = ['age', 'capital.gain', 'capital.loss', 'hours.per.week']

#Instantiate encoder/scaler
ohe    = OneHotEncoder(sparse=False)
le     = LabelEncoder()
mms    = MinMaxScaler()

#To one hot encode the string values, they need to be in a numeric format,
#To do so, we first label encode those features  
w = np.reshape(le.fit_transform(data['workclass']), (30162, 1))
e = np.reshape(le.fit_transform(data['education']), (30162, 1))
m = np.reshape(le.fit_transform(data['marital.status']), (30162, 1))
o = np.reshape(le.fit_transform(data['occupation']), (30162, 1))
r = np.reshape(le.fit_transform(data['race']), (30162, 1))
s = np.reshape(le.fit_transform(data['sex']), (30162, 1))
n = np.reshape(le.fit_transform(data['native.country']), (30162, 1))

#Concatenate the label encoded features
wemorsn = np.concatenate([w, e, m, o, r, s, n], axis=1)

#Scale and encode separate columns
one_hot_encoded_columns = ohe.fit_transform(wemorsn)
label_encoded_columns   = np.reshape(le.fit_transform(data[columns_to_label_encode]), (30162, 1))
min_max_scaled_columns  = mms.fit_transform(data[columns_to_scale])

#Concatenate again
processed_data = np.concatenate([min_max_scaled_columns, one_hot_encoded_columns, label_encoded_columns], axis=1)

#Turn processed data into DataFrame typ
pd_df = pd.DataFrame(processed_data, index=data.index)

pd_df.head(10)

# 5. Model the Data
In this step we want to train different classification models and compare them with each other. We do this in order to select models, which we will examine in more detail in a further step.

But before we train the models, we have to seperate our prediction variable of the features

In [ ]:
from sklearn.model_selection import train_test_split

#Split prediction variable and features
X = pd_df.values[:, :-1]
y = pd_df.values[:, -1]

#Split test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5.1 Model Selection
Eight different prediction models will be compared in their accuracy:
* Decision Tree
* K Nearest Neighbors
* Logistic Regression
* Naive Bayes
* Random Forest
* Support Vector Machine (linear kernel)
* Support Vector Machine (radial basis function kernel)
* Support Vector Machine (sigmoid kernel)

In [ ]:
from sklearn.model_selection import cross_val_score

#Import the classification models
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#Initialize the models
models = []
models.append(('DT', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('LR', LogisticRegression()))
models.append(('RF', RandomForestClassifier()))
models.append(('LSVM', SVC(kernel='linear')))
models.append(('SSVM', SVC(kernel='sigmoid')))
models.append(('RSVM', SVC(kernel='rbf')))

scores = []
names = []

#Set up charts to visualize the results
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(9, 9))
xAxis = 0
yAxis = 0

axes[4, 0].set_title('Mean Accuracy')

print('Mean accuracy on test data:')

#Train and test each model - save the results
for name, model in models:
    model.fit(X_train, y_train)
    score = cross_val_score(model, X_test, y_test, cv=7)
    scores.append(score)
    names.append(name)
    axes[xAxis, yAxis].set_title(name)
    axes[xAxis, yAxis].plot(['1','2','3','4','5','6','7'], score, color='C'+str(len(names)-1))
    axes[xAxis, yAxis].set_xlabel("Validation")
    axes[xAxis, yAxis].set_ylabel("Accuracy")
    axes[4, 0].bar(name, score.mean(), alpha=0.4, color='C'+str(len(names)-1))
    axes[4, 0].set_ylim(0.79, 0.85)
    
    if len(names)%2 == 1:
        yAxis += 1
    else:
        xAxis +=1
        yAxis -=1
    print(name, model.score(X_test, y_test))

#Remove empty chart
axes[4, 1].remove()

fig.tight_layout()
plt.show()

You can see that almost all models have on average more than 80% accuracy in their prediction. Only Gaussian Naives Baiyes model is out of this range with 54%. Furthermore, it can also be seen that some models achieve an accuracy of around 84% during cross validation.

## 5.2 Parameter Tuning

The next step is to use GridSearch to take a closer look at these models and find the right parameters so that their accuracy increases further. The three model types with the best potential are investigated. Logistic Regression, Support Vector Machines and Random Forest. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#Construct pipelines
pipe1 = Pipeline((('LR', LogisticRegression()),))
pipe2 = Pipeline((('SVC', SVC()),))
pipe3 = Pipeline((('RF', RandomForestClassifier()),))

#Define parameters for each pipeline
para1 = {
    'LR__penalty' : ['l1', 'l2'],
    'LR__C': [0.01, 0.1, 1.0, 10]
}
para2 = {
    'SVC__C': [0.1, 1.0, 10],
    'SVC__kernel': ['linear', 'sigmoid', 'rbf'],
    'SVC__gamma': [0.01, 0.1, 1.0]
}
para3 = {
    'RF__n_estimators': [5, 10, 50],
    'RF__max_features': ['auto', 'sqrt', 'log2'],
    'RF__max_depth': [1, 5, 10],
    'RF__min_samples_split': [2, 5, 10],
    'RF__min_samples_leaf': [1, 2, 5]
}


paras = [ para1, para2, para3]
pipes = [ pipe1, pipe2, pipe3]


for i in range(len(pipes)):
    print('GridSearch for model', i)
    grid = GridSearchCV(pipes[i], paras[i], verbose=1, refit=False, n_jobs=-1)
    grid = grid.fit(X_train, y_train)
    print('Finished GridSearch\n')
    print('Best score:', grid.best_score_)
    print(grid.best_params_, '\n\n')

With the GridSearch method, the best parameters for each model could be found. Especially the Random Forest Model benefited from this and achieved an accuracy of more than 85%. 

In the final step, the model with the parameters found must be trained with the entire data set.

In [ ]:
#Initialize the model with the evaluated parameters
RF  = RandomForestClassifier(max_depth=10, max_features='auto', min_samples_leaf=2, min_samples_split=10, n_estimators=10)

#Train the models on the whole data set
RF.fit(X, y)

print('Classification model is ready')

# 6. Conclusion

In summary, what has been achieved so far. A classification model based on the random forest algorithm has been developed to help determine the creditworthiness of individuals. It is important to note that this is only an assistance in this case. Since the data basis allows only a determination as to whether a person earns more or less than 50k, more complex calculations around the granting of credit cannot be carried out. The model could therefore be used in the first step of lending. If a potential lender asks his bank for a loan, the service personnel can determine with the help of personal data that is easy to determine whether the person has a possible creditworthiness and whether it is profitable for the credit institution to process the loan application further. In this case, the service staff could refer the customer to an expert.

The last question in this context is how the credit institution can use the model. Since banks often use software they have developed themselves for security reasons, they can integrate the classification model into their solutions via API. Thus they can also design the interface of the application according to their requirements and link it with other solutions.